# Ideas for the Project

Only Scrape from "Data Scientist" and "Data Analyst" DO NOT include Business Analyst or prefixes like Senior/Junior

## Warning
These functions may be buggy and need fixing but I hope this jumpstarts the data collection process for you. Feel free to edit the functions if they aren't doing what you need properly

In [243]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup 
import nltk
import numpy as np
import re
from nltk.corpus import wordnet
from selenium import webdriver

<Response [200]>

In [300]:
def raw_text(url):
    """Takes a URL as input and performs web scrapping to retrieve the body of the
    webpage (in this case a Linkedin ad)"""
    ad = requests.get(url) #Retrieve webpage
    Html = BeautifulSoup(ad.text, 'html.parser') #Convert html into a nicer format
    text_body = Html.find_all('div', 
                              {'class':"show-more-less-html__markup show-more-less-html__markup--clamp-after-5"})
    text_body = text_body[0].text
    return text_body
def clean_text(doc):
    """Take an unstructured document and tokenize it into a list of words. 
    Then standardize it by lowercasing and lemmatizing each word"""
    words = re.findall(r'(?:[a-zA-Z]|#|"+")+',doc) #Find all alphabetical words (Preserve + and # for C++ and C#)
    clean = [i for i in words if i.isupper() or i.islower()] #Retrieve all words that aren't glued to each other
    dirty = [i for i in words if not i.islower() and not i.isupper()] #Retrieve words stuck together
    dirty = [re.findall('[a-zA-Z][^A-Z]*',i) for i in dirty] #Split all the tangled words ie split 'ThisExample' into ['This','Example']
    clean2 = [j for i in dirty for j in i] #Unlist the list of lists
    words = clean + clean2 #Combine all the words together
    stopwords = nltk.corpus.stopwords.words("english")
    words = [i.lower() for i in words] #Lowercase all words
    words = [i for i in words if i not in stopwords] #Filter out stopwords
    tag_words = nltk.pos_tag(words) #Begin lemmatizing by tagging each word
    tag_words = [(i, wordnet_pos(j)) for (i, j) in tag_words] #Convert the tags into something the lemmatizer understands
    lemmatizer = nltk.WordNetLemmatizer()
    clean_words = [lemmatizer.lemmatize(i, j) for i, j in tag_words] #Lemmatize the words
    #Document should be cleaned up
    return clean_words
def wordnet_pos(tag):
    """Map a Brown POS tag to a WordNet POS tag."""
    
    table = {"N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV, "J": wordnet.ADJ}
    
    # Default to a noun.
    return table.get(tag[0], wordnet.NOUN) #Function created by Bo Ning in Week 6-2
def lang_count(TXT):
    """ Take a body of clean text and count the number of programming languages present"""
    languages = ['python','r','sql','sa','c',
                 'c++','c#','java','javascript',
                 'julia','matlab','swift','tableau'
                'microsoft','github'] #SAS turns into sa after lemmatization
    #ADD MORE LANGUAGES IF NECESSARY
    count = sum([i in TXT for i in languages]) #Check if each language is in the ad
    #And sum the number of programming languages present
    return count
def get_salary(TXT):
    """From a body of raw text, retrieve the salary"""
    salaries = re.findall(r"(\$\d+\,\d+\.\d{1,2})",TXT) #Find all numbers with $ , and .
    if salaries != []:
        return salaries[-1] #Let's work with the maximum salary
    else:
        salaries = re.findall(r"(\$\d+\,\d+)",TXT) 
        if salaries != []:
            return salaries[-1]
        else:
            return "NaN"
def ML_skill(TXT):
    """Using a body of clean text, check whether the words machine learning is present
    to see if it is a required skill"""
    return str(int('machine' in TXT and 'learning' in TXT))
def get_edu(TXT):
    """Using a body of raw text, retrieve the education level"""
    if "Master" in TXT and "Bachelor" in TXT:
        return "4" #Category where Bachlor's is minimum but higher level preferred
    elif "PhD" in TXT:
        return "3"
    elif "Master" in TXT:
        return "2"
    elif "Bachelor" in TXT:
        return "1"
    else:
        return "0" #No education specified
def ad_type(TXT):
    """Using a body of clean text, check whether this is an ad for data analyst or scientists"""
    return str(int("science" in TXT))
def benefits(TXT):
    """Using a body of raw text, check if benefits are included"""
    return str(int('Benefits' in TXT or 'benefits' in TXT))
def exp(TXT):
    """Using a body of raw text, check if experience is required/preferred"""
    sentences = nltk.sent_tokenize(TXT) #Split text into sentences
    years = [re.findall(r"\d+.*year", i) for i in sentences] #Find sentences with years in it
    years = [i for i in years if i != []][0][0] #Get rid of empty values and turn the years of experience into a string
    year = re.findall(r'\d+',years)[0]
    return year

In [382]:
def collect_data(url):
    """Input a URL for a Linkedin Ad and retrieve all relevant data"""
    raw = raw_text(url)
    clean = clean_text(raw)
    return {'Languages':lang_count(clean),
            'Salary':get_salary(raw),
            'Machine Learning':ML_skill(clean),
            'Education':get_edu(raw),
            'Type': ad_type(clean),
            'Benefits':benefits(raw),
            'Experience':exp(raw),
            'url':url}

In [378]:
ex = collect_data(url)

In [362]:
r"(\d+\,\d+)"

[]

In [366]:
re.findall(r"(\$\d+\,\d+)",raw) 

['$30,000', '$45,000']

In [379]:
ex

{'Languages': 1,
 'Salary': '$207,000',
 'Machine Learning': '0',
 'Education': '2',
 'Type': '0',
 'Benefits': '1',
 'Experience': '5'}

In [377]:
url = 'https://www.linkedin.com/jobs/view/3503242054/?alternateChannel=search&refId=rKjt309G%2Bcgm96lG7K9V4w%3D%3D&trackingId=AP31drQeO2%2B7gbjIhTmHOg%3D%3D'

In [373]:
raw_text(url)

"\nDescriptionThe Mars Agency is an award-winning, independently owned global commerce marketing practice with a growth-for-clients focus. With talent spanning the Americas, Europe and Asia, we create breakthrough, connected commerce solutions by balancing the smartest humanity with the latest technology. The agency’s latest MarTech platform, Marilyn®, is the industry's first and only end-to-end commerce advisor.We’re seeking a Senior Data Analyst. The Senior Data Analyst serves plays a critical role in all aspects of analytic client engagements. As part of the integrated team, the Senior Data Analyst engages in analytic project planning, preparing and analyzing data, generating reports, and making action-oriented recommendations to help move our clients business forward. The ideal candidate must be able to proactively communicate ideas and opportunities, execute with a high level of accuracy and demonstrate strong project management skillsWhat You’ll Do:Contribute to the development o